<head>
  <meta name="author" content="Rogério de Oliveira">
  <meta institution="author" content="ITM">
</head>

<img src="https://maua.br/images/selo-60-anos-maua.svg" width=300, align="right">
<!-- <h1 align=left><font size = 6, style="color:rgb(200,0,0)"> optional title </font></h1> -->




# **P1 - RA ÍMPAR**

In [ ]:
#@markdown Nome completo e RA (*\<nome\>, \<RA\>*)
Aluno = 'João Paulo de Souza Rodrigues, 21018090' #@param {type:"string"}






In [ ]:
path = 'https://github.com/Rogerio-mack/work/raw/main/data/'

# **Case**: Airbnb

Neste case, analise dados sobre imóveis alugados pela plataforma Airbnb em NYC no ano de 2019.


In [ ]:
import pandas as pd

df = pd.read_csv(path + 'AB_NYC_2019.csv')
df.head()

id                                            name  host_id  \
0   20913              Charming 1 bed GR8 WBurg LOCATION!    79402   
1   25235               Large 2 Bedroom Great for Groups!    87773   
2   76761  Eveland the Place to Stay & Enjoy a 5-⭐️ 2bdrm   118971   
3  135465             Garden apartment close to Manhattan   663879   
4  169002                Modern Space in Charming Pre-war   805344   

     host_name neighbourhood_group       neighbourhood  latitude  longitude  \
0   Christiana            Brooklyn        Williamsburg  40.70984  -73.95775   
1       Shelly            Brooklyn  Bedford-Stuyvesant  40.68373  -73.92377   
2       Evelyn            Brooklyn         South Slope  40.66552  -73.99019   
3  Christopher            Brooklyn         Fort Greene  40.68626  -73.97598   
4         Alec           Manhattan              Harlem  40.82411  -73.94934   

         room_type  price  minimum_nights  number_of_reviews last_review  \
0  Entire home/apt    100               5                168  2018-07-22   
1  Entire home/apt    125              90                162  2019-06-28   
2  Entire home/apt    169               2                398  2019-06-28   
3  Entire home/apt    170               3                 86  2019-06-20   
4     Private room     65               2                 41  2019-06-16   

   reviews_per_month  calculated_host_listings_count  availability_365  
0               1.57                               1                 0  
1               1.46                               2               137  
2               3.97                               3               182  
3               0.91                               2               286  
4               0.43                               2                59

# Q1. Proprietários com mais imóveis na plataforma

Obtenha os 3 proprietários ("hosts") com mais imóveis sendo anunciados na plataforma, e a respectiva quantidade de imóveis anunciados.


In [ ]:
df.groupby('host_name').id.count().sort_values(ascending=False).head(3)

host_name
Sonder (NYC)    7
Mark            7
David           6
Name: id, dtype: int64

# Q2B. Preço nos bairros com mais imóveis

 Qual o preço médio dos imóveis, considerando somente os preços dos imóveis cujos bairros (`neighbourhood`) tenham mais de 20 imóveis sendo anunciados?  


In [ ]:
neighbourhoods_real_estate = df.groupby('neighbourhood').agg({'id':'count', 'price': 'mean'})

neighbourhoods_real_estate = neighbourhoods_real_estate.sort_values(by='id', ascending=False)
neighbourhoods_real_estate_20 = neighbourhoods_real_estate[neighbourhoods_real_estate.id > 20]

preco_medio = neighbourhoods_real_estate_20.price.mean()

print(f'O preço médio dos imóveis em bairros com mais de 20 anúncios é: USD {preco_medio:.2f}')

O preço médio dos imóveis em bairros com mais de 20 anúncios é: USD 160.21


# Q3B.  Brooklyn *outliers*

Qual o percentual de imóveis de Manhattan com preços estatisticamente discrepantes (*outliers*) com relação aos preços de todos os imóveis?



In [ ]:
neighbourhoods_manhattan = df[df.neighbourhood_group == 'Manhattan']

Q1 = df.price.quantile(0.25)
Q3 = df.price.quantile(0.75)
IQR = Q3 - Q1

threshold = 1.5
outliers = neighbourhoods_manhattan[(neighbourhoods_manhattan['price'] < Q1 - threshold * IQR) | (neighbourhoods_manhattan['price'] > Q3 + threshold * IQR)]

perc_Manhattan = (len(outliers) / len(neighbourhoods_manhattan)) * 100

print( f'Percentuais de outliers de preços, Manhattan: {perc_Manhattan:.2f} %')

Percentuais de outliers de preços, Manhattan: 10.05 %


# Q4B. Scores

Nesta tarefa você deve atribuir a cada imóvel um valor de *score* (`df['score']`) baseado nas seguintes regras:

1. Imóveis com `availability_365` > 90 recebem +10 pontos enquanto imóveis com `availability_365` < 5 recebem -5 pontos;
2. Imóveis com o número de reviews >= 10 recebem +5 pontos para cada 10 reviews (por ex. se número de reviews = 20, são atribuídos +10 pontos). Se número de reviews = 0 o imóvel recebe -5 pontos;
3. Imóveis com preço abaixo da média de seu `neighbourhood`	recebem +5 pontos.

Implemente empregando `apply()` ou um laço de programa, como preferir.

In [ ]:
neighbourhood_price_mean = df.groupby('neighbourhood')['price'].mean()

def calcular_score(row):
    score = 0
    if row['availability_365'] > 90:
        score += 10
    elif row['availability_365'] < 5:
        score -= 5
    if row['number_of_reviews'] >= 10:
        score += (row['number_of_reviews'] // 10) * 5
    elif row['number_of_reviews'] == 0:
        score -= 5

    price_mean = neighbourhood_price_mean[row['neighbourhood']]
    if row['price'] < price_mean:
        score += 5
    return score

df['score'] = df.apply(calcular_score, axis=1)


print(f'A soma dos valores de score é:', df['score'].sum())


A soma dos valores de score é: 10955


# Q5. Enriquecendo os dados

Enriqueça os dados dos imóveis com os valores dos impostos e multas municipais. Cacule a média de impostos e multas de cada grupo de vizinhaça, `neighbourhood_group`. Note que existem imóveis sem impostos e sem multas, esses são imóveis isentos, mas devem ser considerados nos cálculos de média (por exemplo, você pode considerar esses imóveis com valor 0).

In [ ]:
df_taxes = pd.read_csv(path + 'AB_NYC_2019_taxes.csv')
df_taxes['value'] = df_taxes['value'].fillna(0)
# print(df_taxes)

df_taxes_fines = df_taxes.pivot_table(index='id', columns='type', values='value', aggfunc='sum')
# print(df_taxes_fines)

df_merged = pd.merge(df, df_taxes_fines, on='id', how='left')
# print(df_merged.head(1))

df_neighbourhood_taxes = df_merged.groupby('neighbourhood_group').agg({'fines' : 'mean', 'taxes': 'mean'})
df_neighbourhood_taxes['fines_and_taxes'] = df_neighbourhood_taxes['fines'] + df_neighbourhood_taxes['taxes']

print(df_neighbourhood_taxes)

                          fines        taxes  fines_and_taxes
neighbourhood_group                                          
Bronx                126.473684   851.052632       977.526316
Brooklyn             314.472313   858.276873      1172.749186
Manhattan            314.980226   888.844633      1203.824859
Queens               322.769231   804.166667      1126.935897
Staten Island        581.333333  1157.500000      1738.833333


# Q6. Quebras

Construa no `Pandas` um "relatório" (`df_report`) como abaixo. Os valores são ordenados `neighbourhood_group`,`neighbourhood`. Cada "página" contém apenas 30 linhas. A cada "quebra" de `neighbourhood_group` é gerada uma linha com preço médio do `neighbourhood_group` e, então, é iniciada uma nova página.

O item `line_id` é opcional mas deve auxiliá-lo nessa tarefa.

Ao final produza a soma dos preços das páginas abaixo...

In [ ]:
df_groupby_neighbourhood = df.groupby(['neighbourhood']).agg({'neighbourhood_group' : 'first', 'price': 'mean'})
df_groupby_neighbourhood = df_groupby_neighbourhood.sort_values(by=['neighbourhood_group'])
df_groupby_neighbourhood['page_id'] = 1
df_groupby_neighbourhood['line_id'] = 1

for index, row in df_groupby_neighbourhood.iterrows():
  lineId = row['line_id']
  pageId = row['page_id']
  if (lineId == 30):
    df_groupby_neighbourhood.at[index, 'page_id'] = pageId + 1
    df_groupby_neighbourhood.at[index, 'line_id'] = 1
  elif (df_groupby_neighbourhood.at[index, 'neighbourhood_group'] != df_groupby_neighbourhood.at[index + 1, 'neighbourhood_group']):
    df_groupby_neighbourhood.at[index, 'page_id'] = pageId + 1
    df_groupby_neighbourhood.at[index, 'line_id'] = 1
  else:
    df_groupby_neighbourhood.at[index, 'line_id'] = lineId + 1

df_groupby_neighbourhood = df_groupby_neighbourhood.sort_values(by=['page_id', 'line_id'])
print(df_groupby_neighbourhood)

# print(df_groupby_neighbourhood)

# sum_price_pages = df_report[ df_report.page_id.isin([1,24,25,26,28]) ].groupby('page_id').price.sum().astype(int).sum()

# print(f'A soma dos preços das páginas 1, 24, 25 e 28 é: {sum_price_pages}')

ValueError: invalid literal for int() with base 10: 'Mott Haven'

#### Aqui um *sample* do report.

In [ ]:
df_report.iloc[-10::,:]

neighbourhood_group  neighbourhood       price  page_id  line_id
795              Queens       Woodside   75.000000       28       17
796              Queens       Woodside  120.000000       28       18
797              Queens           MEAN   92.227848       28       19
798       Staten Island  Randall Manor   99.000000       29        1
799       Staten Island    South Beach   20.000000       29        2
800       Staten Island    South Beach  100.000000       29        3
801       Staten Island     St. George  130.000000       29        4
802       Staten Island      Todt Hill  135.000000       29        5
803       Staten Island  Tompkinsville   59.000000       29        6
804       Staten Island           MEAN   89.428571       29        7